In [ ]:
import sys
import os

# Make sure your project root is on PYTHONPATH
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)


from models.resnet50_backbone import Resnet50Backbone
from models.vgg_backbone import VGG19BNBackbone

from src.data_loader import ShanghaiTechDataset
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt

In [ ]:
# ====== Part B evaluation ======


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])
val_dataset_B = ShanghaiTechDataset("../data/ShanghaiTech", part="part_B", split="test_data", transform=transform)
val_loader_B = DataLoader(val_dataset_B, batch_size=1)

for name, Cls in [("resnet50", ResNet50Backbone),
                  ("vgg19_bn", VGG19BNBackbone)]:
    model = Cls().to(device)
    model.load_state_dict(torch.load(f"../models/part_B_{name}_model.pth", map_location=device))
    model.eval()

    mae = mse = 0.0
    with torch.no_grad():
        for img, count_map in val_loader_B:
            img = img.to(device)
            count = count_map.sum(dim=(1,2)).unsqueeze(1).float().to(device)
            out = model(img)
            mae += torch.abs(out - count).item()
            mse += ((out - count)**2).item()

    mae /= len(val_dataset_B)
    rmse = (mse / len(val_dataset_B))**0.5
    print(f"Part B — {name}: MAE = {mae:.3f}, RMSE = {rmse:.3f}")